In [1]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import matplotlib.pyplot as plt
import random

import cornac
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality
from cornac.data.text import BaseTokenizer
from cornac.data import Reader
from cornac.metrics import MAE,RMSE,MSE,FMeasure,Precision,Recall,NDCG,NCRR,MRR,AUC,MAP

FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.


In [2]:
# citeulike dataset
docs_citeulike, item_ids_citeulike = cornac.datasets.citeulike.load_text()
feedback_citeulike = cornac.datasets.citeulike.load_feedback(reader=Reader(item_set=item_ids_citeulike))
item_text_modality = TextModality(
    corpus=docs_citeulike,
    ids=item_ids_citeulike,
    tokenizer=BaseTokenizer(sep=" ", stop_words="english"),
    max_vocab=8000,
    max_doc_freq=0.5,
)
# Define an evaluation method to split feedback into train and test sets
citeulike_ratio_split = RatioSplit(
    data=feedback_citeulike,
    test_size=0.2,
    exclude_unknowns=True,
    item_text=item_text_modality,
    verbose=True,
    seed=123,
    rating_threshold=1,
)

rating_threshold = 1.0
exclude_unknowns = True


/Users/pigr/cornac/lib/python3.10/site-packages/cornac/data/dataset.py:361: UserWarning: 18 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/Users/pigr/cornac/lib/python3.10/site-packages/cornac/data/dataset.py:361: UserWarning: 5 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 5551
Number of items = 16949
Number of ratings = 168396
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 5444
Number of items = 14146
Number of ratings = 42053
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 5551
Total items = 16949


### k=10

In [4]:
# Instantiate HFT model
hft_k10 = cornac.models.HFT(
    k=10,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=citeulike_ratio_split, models=[hft_k10], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/42053 [00:00<?, ?it/s]

Ranking:   0%|          | 0/5444 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.0000 | 0.0000 | 0.0000 | 0.4922 | 0.0060 | 0.0076 | 0.0318 |  0.0138 |  0.0191 |       0.0036 |    0.0318 |  117.0666 |  31.9716



In [3]:
data1 = {"model":"HFT",
        "data":"citeulike",
        "k":10,
        "topk":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.01,
        "l2_reg":0.001,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.4922,"F1":0.0060,"MAP":0.0076,"MRR":0.0318,"NCRR":0.0138,"NDCG":0.0191,"Precision":0.0036,"Recall":0.0318,
        "category diversity":0}

### k=10, @TOPK=100

In [6]:
# Instantiate HFT model
hft_k10_top100 = cornac.models.HFT(
    k=10,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=100),Precision(k=100),
           Recall(k=100), NDCG(k=100), NCRR(k=100),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=citeulike_ratio_split, models=[hft_k10_top100], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/42053 [00:00<?, ?it/s]

Ranking:   0%|          | 0/5444 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC | F1@100 |    MAP |    MRR | NCRR@100 | NDCG@100 | Precision@100 | Recall@100 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + -------- + -------- + ------------- + ---------- + --------- + --------
HFT | 0.0000 | 0.0000 | 0.0000 | 0.5067 | 0.0040 | 0.0091 | 0.0348 |   0.0158 |   0.0221 |        0.0022 |     0.0393 |  120.2749 |  31.1218



In [4]:
data2 = {"model":"HFT",
        "data":"citeulike",
        "k":10,
        "topk":100,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.01,
        "l2_reg":0.001,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.5067,"F1":0.0040,"MAP":0.0091,"MRR":0.0348,"NCRR":0.0158,"NDCG":0.0221,"Precision":0.0022,"Recall":0.0393,
        "category diversity":0}

### k=50

In [9]:
# Instantiate HFT model
hft_k50 = cornac.models.HFT(
    k=50,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=citeulike_ratio_split, models=[hft_k50], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/42053 [00:00<?, ?it/s]

Ranking:   0%|          | 0/5444 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.0000 | 0.0000 | 0.0000 | 0.4968 | 0.0019 | 0.0027 | 0.0125 |  0.0046 |  0.0060 |       0.0011 |    0.0100 |  299.1378 |  48.3684



In [5]:
data3 = {"model":"HFT",
        "data":"citeulike",
        "k":50,
        "topk":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.01,
        "l2_reg":0.001,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.4968,"F1":0.0019,"MAP":0.0027,"MRR":0.0125,"NCRR":0.0046,"NDCG":0.0060,"Precision":0.0011,"Recall":0.0100,
        "category diversity":0}

### k=100

In [11]:
# Instantiate HFT model
hft_k100 = cornac.models.HFT(
    k=100,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=citeulike_ratio_split, models=[hft_k100], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/42053 [00:00<?, ?it/s]

Ranking:   0%|          | 0/5444 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.0000 | 0.0000 | 0.0000 | 0.5019 | 0.0013 | 0.0020 | 0.0071 |  0.0026 |  0.0039 |       0.0008 |    0.0073 |  390.8251 |  47.0222



In [6]:
data4 = {"model":"HFT",
        "data":"citeulike",
        "k":100,
        "topk":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.01,
        "l2_reg":0.001,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.5019,"F1":0.0013,"MAP":0.0020,"MRR":0.0071,"NCRR":0.0026,"NDCG":0.0039,"Precision":0.0008,"Recall":0.0073,
        "category diversity":0}

### k=200

In [5]:
# Instantiate HFT model
hft_k200 = cornac.models.HFT(
    k=200,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=citeulike_ratio_split, models=[hft_k200], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/42053 [00:00<?, ?it/s]

Ranking:   0%|          | 0/5444 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.0000 | 0.0000 | 0.0000 | 0.4948 | 0.0017 | 0.0067 | 0.0193 |  0.0097 |  0.0099 |       0.0010 |    0.0112 |  377.1846 |  34.9846



In [7]:
data5 = {"model":"HFT",
        "data":"citeulike",
        "k":200,
        "topk":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.01,
        "l2_reg":0.001,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.4948,"F1":0.0017,"MAP":0.0067,"MRR":0.0193,"NCRR":0.0097,"NDCG":0.0099,"Precision":0.0010,"Recall":0.0112,
        "category diversity":0}

### k=300

In [8]:
# Instantiate HFT model
hft_k300 = cornac.models.HFT(
    k=300,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.01,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=citeulike_ratio_split, models=[hft_k300], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/42053 [00:00<?, ?it/s]

Ranking:   0%|          | 0/5444 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.0000 | 0.0000 | 0.0000 | 0.5004 | 0.0038 | 0.0070 | 0.0304 |  0.0127 |  0.0151 |       0.0023 |    0.0224 |  477.0192 |  41.4342



In [9]:
data6 = {"model":"HFT",
        "data":"citeulike",
        "k":300,
        "topk":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.01,
        "l2_reg":0.001,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.5004,"F1":0.0038,"MAP":0.0070,"MRR":0.0304,"NCRR":0.0127,"NDCG":0.0151,"Precision":0.0023,"Recall":0.0224,
        "category diversity":0}

### k=10,lambda_text=0.1

In [10]:
# Instantiate HFT model
hft_lamText01_k10 = cornac.models.HFT(
    k=10,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=0.1,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=citeulike_ratio_split, models=[hft_lamText01_k10], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/42053 [00:00<?, ?it/s]

Ranking:   0%|          | 0/5444 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.0000 | 0.0000 | 0.0000 | 0.4987 | 0.0016 | 0.0055 | 0.0208 |  0.0090 |  0.0088 |       0.0010 |    0.0099 |  155.6609 |  28.4342



In [11]:
data7 = {"model":"HFT",
        "data":"citeulike",
        "k":10,
        "topk":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.1,
        "l2_reg":0.001,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.4987,"F1":0.0016,"MAP":0.0055,"MRR":0.0208,"NCRR":0.0090,"NDCG":0.0088,"Precision":0.0010,"Recall":0.0099,
        "category diversity":0}

### k=10, lambda_text=1

In [12]:
# Instantiate HFT model
hft_lamText1_k10 = cornac.models.HFT(
    k=10,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.001,
    lambda_text=1,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=citeulike_ratio_split, models=[hft_lamText1_k10], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/42053 [00:00<?, ?it/s]

Ranking:   0%|          | 0/5444 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.0000 | 0.0000 | 0.0000 | 0.5024 | 0.0006 | 0.0009 | 0.0028 |  0.0006 |  0.0011 |       0.0004 |    0.0024 |  212.4834 |  31.4043



In [13]:
data8 = {"model":"HFT",
        "data":"citeulike",
        "k":10,
        "topk":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":1,
        "l2_reg":0.001,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.5024,"F1":0.0006,"MAP":0.0009,"MRR":0.0028,"NCRR":0.0006,"NDCG":0.0011,"Precision":0.0004,"Recall":0.0024,
        "category diversity":0}

### k=10, lambda_text=0.1, l2_reg=0.01

In [14]:
# Instantiate HFT model
hft_l2001_lamText01_k10 = cornac.models.HFT(
    k=10,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.01,
    lambda_text=0.1,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=citeulike_ratio_split, models=[hft_l2001_lamText01_k10], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/42053 [00:00<?, ?it/s]

Ranking:   0%|          | 0/5444 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.0000 | 0.0000 | 0.0000 | 0.4960 | 0.0015 | 0.0047 | 0.0185 |  0.0077 |  0.0076 |       0.0009 |    0.0090 |  159.6241 |  35.9886



In [15]:
data9 = {"model":"HFT",
        "data":"citeulike",
        "k":10,
        "topk":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.1,
        "l2_reg":0.01,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.4960,"F1":0.0015,"MAP":0.0047,"MRR":0.0185,"NCRR":0.0077,"NDCG":0.0076,"Precision":0.0009,"Recall":0.0090,
        "category diversity":0}

### k=10, lambda_text=0.1, l2_reg=0.1

In [16]:
# Instantiate HFT model
hft_l201_lamText01_k10 = cornac.models.HFT(
    k=10,
    max_iter=10,
    grad_iter=5,
    l2_reg=0.1,
    lambda_text=0.1,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=citeulike_ratio_split, models=[hft_l201_lamText01_k10], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/42053 [00:00<?, ?it/s]

Ranking:   0%|          | 0/5444 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.0000 | 0.0000 | 0.0000 | 0.4977 | 0.0016 | 0.0057 | 0.0214 |  0.0094 |  0.0092 |       0.0009 |    0.0102 |  156.7292 |  29.1349



In [17]:
data10 = {"model":"HFT",
        "data":"citeulike",
        "k":10,
        "topk":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.1,
        "l2_reg":0.1,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.4977,"F1":0.0016,"MAP":0.0057,"MRR":0.0214,"NCRR":0.0094,"NDCG":0.0092,"Precision":0.0009,"Recall":0.0102,
        "category diversity":0}

### k=10. lambda_text=0.1, l2_reg=1

In [18]:
# Instantiate HFT model
hft_l21_lamText01_k10 = cornac.models.HFT(
    k=10,
    max_iter=10,
    grad_iter=5,
    l2_reg=1,
    lambda_text=0.1,
    vocab_size=8000,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=citeulike_ratio_split, models=[hft_l21_lamText01_k10], metrics=metrics, user_based=False
).run()


[HFT] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[HFT] Evaluation started!


Rating:   0%|          | 0/42053 [00:00<?, ?it/s]

Ranking:   0%|          | 0/5444 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
HFT | 0.0000 | 0.0000 | 0.0000 | 0.4933 | 0.0016 | 0.0060 | 0.0222 |  0.0098 |  0.0095 |       0.0010 |    0.0103 |  160.2646 |  29.1127



In [19]:
data11 = {"model":"HFT",
        "data":"citeulike",
        "k":10,
        "topk":50,
        "max_iter":10,
        "grad_iter":5,
        "lambda_text":0.1,
        "l2_reg":1,
        "MAE":0.0000,"MSE":0.0000,"RMSE":0.0000,"AUC":0.4933,"F1":0.0016,"MAP":0.0060,"MRR":0.0222,"NCRR":0.0098,"NDCG":0.0095,"Precision":0.0010,"Recall":0.0103,
        "category diversity":0}

In [20]:
data = [data1,data3,data4,data5,data6,data7,data8,data9,data10,data11]
df = pd.DataFrame(columns = data1.keys())
i=0
for d in data:
    df = pd.concat([df, pd.DataFrame(d,columns = list(d.keys()),index=[i])], ignore_index=False)
    i += 1

In [21]:
df

,model,data,k,topk,max_iter,grad_iter,lambda_text,l2_reg,MAE,MSE,RMSE,AUC,F1,MAP,MRR,NCRR,NDCG,Precision,Recall,category diversity
0,HFT,citeulike,10,50,10,5,0.01,0.001,0.0,0.0,0.0,0.4922,0.0060,0.0076,0.0318,0.0138,0.0191,0.0036,0.0318,0
1,HFT,citeulike,50,50,10,5,0.01,0.001,0.0,0.0,0.0,0.4968,0.0019,0.0027,0.0125,0.0046,0.0060,0.0011,0.0100,0
2,HFT,citeulike,100,50,10,5,0.01,0.001,0.0,0.0,0.0,0.5019,0.0013,0.0020,0.0071,0.0026,0.0039,0.0008,0.0073,0
3,HFT,citeulike,200,50,10,5,0.01,0.001,0.0,0.0,0.0,0.4948,0.0017,0.0067,0.0193,0.0097,0.0099,0.0010,0.0112,0
4,HFT,citeulike,300,50,10,5,0.01,0.001,0.0,0.0,0.0,0.5004,0.0038,0.0070,0.0304,0.0127,0.0151,0.0023,0.0224,0
5,HFT,citeulike,10,50,10,5,0.10,0.001,0.0,0.0,0.0,0.4987,0.0016,0.0055,0.0208,0.0090,0.0088,0.0010,0.0099,0
6,HFT,citeulike,10,50,10,5,1.00,0.001,0.0,0.0,0.0,0.5024,0.0006,0.0009,0.0028,0.0006,0.0011,0.0004,0.0024,0
7,HFT,citeulike,10,50,10,5,0.10,0.010,0.0,0.0,0.0,0.4960,0.0015,0.0047,0.0185,0.0077,0.0076,0.0009,0.0090,0
8,HFT,citeulike,10,50,10,5,0.10,0.100,0.0,0.0,0.0,0.4977,0.0016,0.0057,0.0214,0.0094,0.0092,0.0009,0.0102,0
9,HFT,citeulike,10,50,10,5,0.10,1.000,0.0,0.0,0.0,0.4933,0.0016,0.0060,0.0222,0.0098,0.0095,0.0010,0.0103,0


In [22]:
df.to_csv('/Users/pigr/Desktop/cornac/cornac/dataset/hft_citeulike.csv',header=-1,index=0)